# <h1 align='center'> Xây dựng bộ dữ liệu thống kê các ván đấu theo từng đội tuyển LCK

## <h3 align='justify'> Mục tiêu: Từ bộ dữ liệu ban đầu bao gồm các thống kê theo từng player và đội tuyển, tiến hành trích xuất, xử lý (chỉ bao gồm lọc, lược bỏ các hàng và cột không có ý nghĩa thống kê theo đội).

## <h3 align='justify'> Kết quả: Bộ dữ liệu chứa các thống kê theo từng ván đấu của các đội tuyển ở các giải đấu lớn, hấp dẫn nhất trong năm 2024.



In [1]:
import pandas as pd

# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)        # Show all rows
pd.set_option('display.max_columns', None)     # Show all columns
pd.set_option('display.width', None)           # Auto-detect display width
pd.set_option('display.max_colwidth', None)    # Show full content in each cell


In [2]:
#Lấy vào bộ dataset gốc ban đầu
df = pd.read_csv("../Data/2024_LoL_esports_match_data_from_OraclesElixir.csv", low_memory=False)


In [3]:
#Giữ lại các thống kê ván đấu ở các giải 
df = df[df["league"].isin(["LCK", "LPL", "LEC", "LCS", "VCS", "PCS", "LLA", "CBLOL", "WLDs", "EWC", 'MSI'])]

df = df[df["position"] == "team"]

In [5]:
#Thống kê giá trị khuyết
# Calculate missing values count and percentage
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Combine into a DataFrame for better visualization
missing_stats = pd.DataFrame({
    'Missing Values': missing_values,
    'Missing Percentage': missing_percentage
})

# Sort by missing values in descending order
missing_stats = missing_stats[missing_stats['Missing Values'] > 0].sort_values('Missing Values', ascending=False)

# Display statistics for columns with missing values
print("Columns with missing values:")
print(missing_stats)

# Get total number of missing values
total_missing = df.isnull().sum().sum()
print(f"\nTotal number of missing values: {total_missing}")

# Get number of columns with missing values
cols_with_missing = len(missing_stats)
print(f"Number of columns with missing values: {cols_with_missing}")

# Get percentage of total missing values
total_cells = df.size
total_missing_percentage = (total_missing / total_cells) * 100
print(f"Percentage of total missing values: {total_missing_percentage:.2f}%")

missing_stats.to_excel("missing_stats.xlsx")

Columns with missing values:
                          Missing Values  Missing Percentage
damageshare                         5876          100.000000
earnedgoldshare                     5876          100.000000
playername                          5876          100.000000
playerid                            5876          100.000000
champion                            5876          100.000000
total cs                            5876          100.000000
firstbloodvictim                    5876          100.000000
firstbloodassist                    5876          100.000000
firstbloodkill                      5876          100.000000
dragons (type unknown)              4260           72.498298
monsterkillsownjungle               4260           72.498298
monsterkillsenemyjungle             4260           72.498298
url                                 4260           72.498298
golddiffat25                        1808           30.769231
opp_xpat25                          1808           30.76

Đầu tiên, ta sẽ drop các cột có thống kê dữ liệu khuyết là 100%, vì điều này chứng tỏ cột không có ý nghĩa thống kê với đội; sau đó đến các cột khuyết trên 70%, vì các cột này khó có thể điền khuyết khi thiếu nhiều dữ liệu tới vậy.

In [5]:
# Lấy các cột khuyết 100%
columns_to_drop = missing_stats[missing_stats['Missing Percentage'] == 100].index.tolist()

# Drop cột khuyết 100%
df = df.drop(columns=columns_to_drop)

# Drop các cột khuyết trên 70%
df = df.drop(['url', 'dragons (type unknown)', 'monsterkillsenemyjungle','monsterkillsownjungle'], axis = 1)

# Verify the drop by checking missing values again
missing_values_after = df.isnull().sum()
missing_percentage_after = (df.isnull().sum() / len(df)) * 100

# Create summary DataFrame
missing_stats_after = pd.DataFrame({
    'Missing Values': missing_values_after,
    'Missing Percentage': missing_percentage_after
})

# Show only columns with missing values, sorted by percentage
missing_stats_after = missing_stats_after[
    missing_stats_after['Missing Values'] > 0
].sort_values('Missing Values', ascending=False)

print("Columns with missing values after dropping 100% missing columns:")
print(missing_stats_after)





Columns with missing values after dropping 100% missing columns:
                          Missing Values  Missing Percentage
opp_deathsat25                      1808           30.769231
xpdiffat25                          1808           30.769231
goldat25                            1808           30.769231
xpat25                              1808           30.769231
csat25                              1808           30.769231
opp_goldat25                        1808           30.769231
opp_csat25                          1808           30.769231
golddiffat25                        1808           30.769231
opp_xpat25                          1808           30.769231
csdiffat25                          1808           30.769231
killsat25                           1808           30.769231
assistsat25                         1808           30.769231
deathsat25                          1808           30.769231
opp_killsat25                       1808           30.769231
opp_assistsat25     

Tiến hành "fill" các giá trị null nhưng thực chất không phải là "null", rơi vào các trường hợp sau: 
- **'split':** Các dữ liệu khuyết ở cột này là do ở 3 giải đấu MSI, EWC và Worlds không có chia split, thuộc tính này chỉ có ở các giải đấu khu vực -> Tiến hành điền khuyết bằng giá trị "None".
- **Các cột thống kê theo thời gian thực trong ván đấu:** Ở đây ta sẽ phải làm rõ dữ liệu khuyết ở các cột này thật sự khuyết hay là do ván đấu kết thúc sớm hơn các mốc thời gian này, dựa vào cột **'datacompleteness'**. Từ đó sẽ điền 0 cho các ván đấu kết thúc sớm hơn các mốc thời gian này; và giữ nguyên khuyết cho các giá trị khuyết. 

'split'

In [6]:
#Thống kê các giá trị xuất hiện trong thuộc tính split

# Thống kê các giá trị xuất hiện trong thuộc tính split
split_counts = df['split'].value_counts()
split_percentages = df['split'].value_counts(normalize=True) * 100

# Tạo DataFrame để hiển thị kết quả
split_stats = pd.DataFrame({
    'Số lượng': split_counts,
    'Tỷ lệ %': split_percentages.round(2)
})

print("Thống kê các giá trị trong thuộc tính 'split':")
print("-" * 50)
print(split_stats)

# Kiểm tra giá trị null nếu có
null_count = df['split'].isnull().sum()
if null_count > 0:
    print(f"\nSố lượng giá trị null: {null_count}")
    print(f"Tỷ lệ null: {(null_count/len(df)*100):.2f}%")

Thống kê các giá trị trong thuộc tính 'split':
--------------------------------------------------
                   Số lượng  Tỷ lệ %
split                               
Spring                 2158    39.83
Summer                 1814    33.48
Split 2                 266     4.91
Split 1                 260     4.80
Summer Placements       260     4.80
Opening                 248     4.58
Closing                 178     3.29
Winter                  170     3.14
Finals                   64     1.18

Số lượng giá trị null: 458
Tỷ lệ null: 7.79%


In [7]:
# Điền khuyết cho cột split bằng 'None'
df['split'] = df['split'].fillna('None')

Các cột thống kê theo thời gian thực trong ván đấu

In [8]:
# Xác định các cột thống kê theo thời gian
time_stats_columns = [col for col in df.columns if any(marker in col for marker in ['at10', 'at15', 'at20','at25'])]

# In ra các cột được tìm thấy để kiểm tra
print("Các cột thống kê theo thời gian:")
print("\n".join(time_stats_columns))

Các cột thống kê theo thời gian:
goldat10
xpat10
csat10
opp_goldat10
opp_xpat10
opp_csat10
golddiffat10
xpdiffat10
csdiffat10
killsat10
assistsat10
deathsat10
opp_killsat10
opp_assistsat10
opp_deathsat10
goldat15
xpat15
csat15
opp_goldat15
opp_xpat15
opp_csat15
golddiffat15
xpdiffat15
csdiffat15
killsat15
assistsat15
deathsat15
opp_killsat15
opp_assistsat15
opp_deathsat15
goldat20
xpat20
csat20
opp_goldat20
opp_xpat20
opp_csat20
golddiffat20
xpdiffat20
csdiffat20
killsat20
assistsat20
deathsat20
opp_killsat20
opp_assistsat20
opp_deathsat20
goldat25
xpat25
csat25
opp_goldat25
opp_xpat25
opp_csat25
golddiffat25
xpdiffat25
csdiffat25
killsat25
assistsat25
deathsat25
opp_killsat25
opp_assistsat25
opp_deathsat25


In [9]:
# Hàm điền giá trị dựa vào điều kiện
def fill_time_based_stats(row, column):
    # Lấy thời gian từ tên cột một cách an toàn hơn
    for time in [10, 15, 20, 25]:
        if f"at{time}" in column:
            time_marker = time
            break
    else:
        return row[column]  # Trả về giá trị gốc nếu không tìm thấy mốc thời gian
    
    # Nếu gamelength < time_marker và datacompleteness là 'complete'
    # thì điền 0 (trận đấu kết thúc trước mốc thời gian)
    if pd.isna(row[column]) and row['gamelength'] < time_marker * 60 and row['datacompleteness'] == 'complete':
        return 0
    return row[column]

In [10]:
# Áp dụng điều kiện cho từng cột
for col in time_stats_columns:
    df[col] = df.apply(lambda row: fill_time_based_stats(row, col), axis=1)

# Kiểm tra kết quả
for col in time_stats_columns:
    missing_count = df[col].isnull().sum()
    total_count = len(df)
    print(f"\nCột {col}:")
    print(f"- Số giá trị còn thiếu: {missing_count}")
    print(f"- Tỷ lệ thiếu: {(missing_count/total_count*100):.2f}%")


Cột goldat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột xpat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột csat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột opp_goldat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột opp_xpat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột opp_csat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột golddiffat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột xpdiffat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột csdiffat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột killsat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột assistsat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột deathsat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột opp_killsat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột opp_assistsat10:
- Số giá trị còn thiếu: 1616
- Tỷ lệ thiếu: 27.50%

Cột opp_deathsat10:
- Số giá trị còn thiếu: 1

In [11]:
# Verify the drop by checking missing values again
missing_values_after = df.isnull().sum()
missing_percentage_after = (df.isnull().sum() / len(df)) * 100

# Create summary DataFrame
missing_stats_after = pd.DataFrame({
    'Missing Values': missing_values_after,
    'Missing Percentage': missing_percentage_after
})

# Show only columns with missing values, sorted by percentage
missing_stats_after = missing_stats_after[
    missing_stats_after['Missing Values'] > 0
].sort_values('Missing Values', ascending=False)

print("Columns with missing values after dropping 100% missing columns:")
print(missing_stats_after)


Columns with missing values after dropping 100% missing columns:
                          Missing Values  Missing Percentage
assistsat10                         1616           27.501702
opp_csat20                          1616           27.501702
opp_goldat20                        1616           27.501702
csat20                              1616           27.501702
xpat20                              1616           27.501702
goldat20                            1616           27.501702
opp_deathsat15                      1616           27.501702
opp_assistsat15                     1616           27.501702
opp_killsat15                       1616           27.501702
deathsat15                          1616           27.501702
assistsat15                         1616           27.501702
killsat15                           1616           27.501702
csdiffat15                          1616           27.501702
xpdiffat15                          1616           27.501702
golddiffat15        

- Từ hiểu biết về các giá trị trong bộ dữ liệu cũng như cách mà tác giả thu thập dữ liệu, bọn em biết rằng các cột có số lượng giá trị khuyết là 1616 giá trị đến từ các trận đấu được diễn ra trên server giải đấu của Trung Quốc (Vì chính sách bảo mật dữ liệu của nước này nên các dữ liệu chi tiết hơn không thể được thu thập -> Nguyên nhân gây khuyết giá trị). Tạm thời sẽ giữ nguyên các giá trị khuyết ấy, có thể xem xét hướng giải quyết khi bước vào tiền xử lý dữ liệu. 

- Còn với các cột còn lại như banX, pickX thì số lượng giá trị khuyết chỉ đếm trên đầu ngón tay. Chúng em quyết định sẽ fill thủ công các giá trị này.

Fill thủ công các cột ban pick

In [12]:
# Lấy ra các cột ban và pick
ban_pick_columns = [col for col in df.columns if col.startswith(('ban', 'pick'))]

# Tạo DataFrame mới chỉ với các thông tin cần thiết từ các hàng có giá trị khuyết
missing_df = df[df[ban_pick_columns].isnull().any(axis=1)][
    ['gameid', 'datacompleteness', 'league', 'date', 'teamname', 'side', 'result'] + ban_pick_columns
]

# Hiển thị thông tin tổng quan
print(f"Số lượng hàng có giá trị khuyết trong ban/pick: {len(missing_df)}")

# Thống kê số lượng giá trị khuyết cho mỗi cột ban/pick
missing_counts = missing_df[ban_pick_columns].isnull().sum()
print("\nSố lượng giá trị khuyết theo từng cột:")
print(missing_counts[missing_counts > 0])

# Xuất ra file CSV để điền khuyết thủ công
missing_df.to_csv('../Data/missing_bans_picks.csv', index=False)

print("\nĐã xuất file CSV thành công!")
print(f"Số lượng hàng được xuất: {len(missing_df)}")


Số lượng hàng có giá trị khuyết trong ban/pick: 27

Số lượng giá trị khuyết theo từng cột:
ban1      1
ban3      1
ban4      4
ban5     11
pick1    10
pick2    10
pick3    10
pick4    10
pick5    10
dtype: int64

Đã xuất file CSV thành công!
Số lượng hàng được xuất: 27


Trong quá trình điền khuyết thủ công cho các cột ban pick, bọn em nhận thấy rằng các giá trị khuyết ở các cột ban không đến từ việc thiếu sót trong quá trình thu thập, mà là do các đội trong quá trình ban/pick đã thật sự bỏ trống/không cấm tướng ở lượt đó. Cho nên bọn em chỉ cần điền khuyết cho phần pick.

In [17]:
# Đọc file CSV đã điền khuyết
filled_df = pd.read_csv('../Data/missing_bans_picks.csv')

# Cập nhật các giá trị trong DataFrame gốc
for index, row in filled_df.iterrows():
    # Lấy gameid và teamname để xác định hàng cần cập nhật
    mask = (df['gameid'] == row['gameid']) & (df['teamname'] == row['teamname'])
    
    # Cập nhật các cột ban/pick
    for col in ban_pick_columns:
        df.loc[mask, col] = row[col]

# Kiểm tra lại kết quả
# Đếm số lượng giá trị null còn lại trong các cột ban/pick
remaining_nulls = df[ban_pick_columns].isnull().sum()
print("\nKiểm tra số lượng giá trị null còn lại trong các cột ban/pick:")
print(remaining_nulls[remaining_nulls > 0])


Kiểm tra số lượng giá trị null còn lại trong các cột ban/pick:
ban1     1
ban3     1
ban4     4
ban5    11
dtype: int64


Sau khi check lại, tiến hành drop 1 số cột có ý nghĩa thống kê như nhau.

In [15]:
df = df.drop(['teamkills','teamdeaths'], axis = 1)

KeyError: "['teamkills', 'teamdeaths'] not found in axis"

Tới đây là xong phần xây dựng bộ dữ liệu thống kê thông số trong mỗi ván đấu theo mỗi đội.

In [16]:
#Xuất file csv hoàn chỉnh
df.to_csv('../Data/Team_stats_Tournamennts.csv', index=False)

Xuất ra file của riêng giải LCK

In [ ]:
lck_df = df[df['league'] == 'LCK']

lck_df.to_csv('../Data/LCK_Tournament.csv')